In [339]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
from functions import *
from sort import Sort

In [340]:
def detect_red_hough(board):
    img_hsv=cv2.cvtColor(board, cv2.COLOR_RGB2HSV)

    # lower mask (0-10)
    lb = 120
    lower_red = np.array([0,lb,lb])
    upper_red = np.array([4,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([160,lb,lb])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

    k1 = np.ones((5,5))

    # join my masks
    redmask = mask0+mask1
    redmask = cv2.erode(redmask, k1)
    #redmask = cv2.erode(redmask, k1)
    #redmask = cv2.erode(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)


    #redmask = cv2.dilate(redmask, k2)
    #redmask = cv2.dilate(redmask, k2)

    redboard = np.array(board)
    redboard[redmask==0] = 0
    redboard = cv2.cvtColor(redboard, cv2.COLOR_RGB2GRAY)
    redboard = cv2.equalizeHist(redboard)

    # redboard = cv2.cvtColor(redboard, cv2.COLOR_RGB2GRAY)

    minDist = 30
    param1 = 80 #canie threshold
    param2 = 28 #200 #smaller value-> more false circles
    minRadius = 30
    maxRadius = 40 #10

    circles = cv2.HoughCircles(redboard,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    return circles,redboard

# def detect_white_hough(board):
#     epsilon = 90 #190
#     white_upper_range = np.array([255,255,255])
#     white_lower_range = white_upper_range - epsilon

#     k1 = np.ones((15,15))
#     k2 = np.ones((7,7))
    

#     whitemask = cv2.inRange(board, white_lower_range, white_upper_range)
#     whitemask = cv2.erode(whitemask, k1)
#     whitemask = cv2.erode(whitemask, k1)


#     #whitemask = cv2.dilate(whitemask ,k)
#     #whitemask = cv2.dilate(whitemask ,k)
#     whitemask = cv2.dilate(whitemask, k2)
#     whitemask = cv2.dilate(whitemask, k2)

#     whiteboard = np.array(board)
#     whiteboard[whitemask==0] = 0
#     whiteboard = cv2.cvtColor(whiteboard, cv2.COLOR_RGB2GRAY)

#     minDist = 30
#     param1 = 30 #500
#     param2 = 11 #30 #smaller value-> more false circles
#     minRadius = 30
#     maxRadius = 40 #10

#     #circles = cv2.HoughCircles(whiteboard,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
#     circles = cv2.HoughCircles(whitemask,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
#     return circles,whitemask


def detect_white_hough(board):
    epsilon = 34 #34 with equalization
    white_upper_range = np.array([255])
    white_lower_range = white_upper_range - epsilon

    k1 = np.ones((11,11))
    k2 = np.ones((7,7))
    
    board_ = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)
    board_ = cv2.equalizeHist(board_)

    whitemask = cv2.inRange(board_, white_lower_range, white_upper_range)
    whitemask = cv2.erode(whitemask, k1)
    #whitemask = cv2.erode(whitemask, k1)
    #whitemask = cv2.erode(whitemask, k2)

    whitemask = cv2.dilate(whitemask,k2)
    whitemask = cv2.dilate(whitemask,k2)
    whitemask = cv2.dilate(whitemask, k2)
    whitemask = cv2.dilate(whitemask, k2)

    whiteboard = np.array(board)
    whiteboard[whitemask==0] = 0
    whiteboard = cv2.cvtColor(whiteboard, cv2.COLOR_RGB2GRAY)

    minDist = 30
    param1 = 60 #60
    param2 = 25 #30 #smaller value-> more false circles
    minRadius = 30
    maxRadius = 40 #10

    circles = cv2.HoughCircles(whiteboard,cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    #circles = cv2.HoughCircles(whitemask,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    return circles,whiteboard


In [341]:
cap = cv2.VideoCapture('videos/test1.mp4')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (600,  600))
out2 = cv2.VideoWriter('output_mask.mp4', fourcc, 30.0, (600,  600))

red_tracker = Sort(max_age=120, iou_threshold=0.1, min_hits=6)
white_tracker = Sort(max_age=120, iou_threshold=0.1, min_hits=6)

j = 0
 
while cap.isOpened():
    j+=1
    ret, frame = cap.read()

    if j < 350:
        continue
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    frame = cv2.resize(frame, (1000,1000))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    red_pawns, redboard = detect_red_hough(frame)
    red_detections = [[x-35, y-35, x + 35, y + 35, 1] for x, y, r in red_pawns[0, :]]

    red_tracked = red_tracker.update(np.array(red_detections))

    for obj in red_tracked:
        obj = np.uint16(np.around(obj))
        x_min, y_min, x_max, y_max, obj_id = obj

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(frame, f'ID: {int(obj_id)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    white_pawns, whitemask = detect_white_hough(frame)
    white_detections = [[x-35, y-35, x + 35, y + 35, 1] for x, y, r in white_pawns[0, :]]

    white_tracked = white_tracker.update(np.array(white_detections))

    for obj in white_tracked:
        obj = np.uint16(np.around(obj))
        x_min, y_min, x_max, y_max, obj_id = obj

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)
        cv2.putText(frame, f'ID: {int(obj_id)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)



    # plt.imshow(frame)
    # plt.show()

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame = cv2.resize(frame, (600,600))
    out.write(frame)

    whitemask = cv2.resize(whitemask, (600,600))
    whitemask = cv2.cvtColor(whitemask, cv2.COLOR_GRAY2BGR)
    out2.write(whitemask)

    if j >= 450:
        break
 
print('done')
cap.release()
out.release()
out2.release()

done
